In [10]:
%pip install numpy
%pip install pandas
%pip install tqdm
%pip install scikit-learn
%pip install catboost


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor
import pickle

pd.set_option('display.max_columns', 200)

In [3]:
X = pd.read_csv("X.csv")
y = pd.read_csv("y.csv")
X['ZTAMB'] = 40 / 21 * (X['ZTAMB'] - 7.5)

In [4]:
#function for counting nan values in cols
def count_nan_cols(X : pd.DataFrame):
  count = 0
  for i in X.columns:
    for j in X[i]:
      if pd.isna(j):
        count+=1
    if count != 0:
      print(i, ':', count)
    count = 0

In [5]:
merged = X.merge(y)
Mt5b = merged[merged['flight_phase'] == 'TAKEOFF']
Mt5b = Mt5b[Mt5b['engine_family'] == 'CFM56-5B']

Mc5b = merged[merged['flight_phase'] == 'CRUISE']
Mc5b = Mc5b[Mc5b['engine_family'] == 'CFM56-5B']

Mt8e = merged[merged['flight_phase'] == 'TAKEOFF']
Mt8e = Mt8e[Mt8e['engine_family'] == 'CF34-8E']

Mc8e = merged[merged['flight_phase'] == 'CRUISE']
Mc8e = Mc8e[Mc8e['engine_family'] == 'CF34-8E']

Mt7 = merged[merged['flight_phase'] == 'TAKEOFF']
Mt7 = Mt7[Mt7['engine_family'] == 'CFM56-7']

Mc7 = merged[merged['flight_phase'] == 'CRUISE']
Mc7 = Mc7[Mc7['engine_family'] == 'CFM56-7']


In [6]:
Mnames = ['Mt5b', 'Mc5b', 'Mt8e', 'Mc8e', 'Mt7', 'Mc7' ]

In [7]:
for i in range(6):
  exec(f"{Mnames[i]}.dropna(axis='columns', thresh=20, inplace=True)")
  exec(f"{Mnames[i]}.dropna(inplace=True)")

In [8]:
col = [0, 2, 3]

Xt5b = Mt5b.iloc[:, :-15]
yt5b = Mt5b.iloc[:, col + [i for i in range(34, 49)]]

Xc5b = Mc5b.iloc[:,:-24]
yc5b = Mc5b.iloc[:,col + [i for i in range(39, 62)]]

Xt8e = Mt8e.iloc[:,:-10]
yt8e = Mt8e.iloc[:,col + [i for i in range(40, 50)]]

Xc8e = Mc8e.iloc[:,:-15]
yc8e = Mc8e.iloc[:,col + [i for i in range(39, 54)]]

Xt7 = Mt7.iloc[:,:-14]
yt7 = Mt7.iloc[:,col + [i for i in range(32, 46)]]

Xc7 = Mc7.iloc[:,:-19]
yc7 = Mc7.iloc[:, col + [i for i in range(29, 48)]]

In [9]:
Xnames = ['Xt5b', 'Xc5b', 'Xt8e', 'Xc8e', 'Xt7', 'Xc7' ]
XObjs = [Xt5b, Xc5b, Xt8e, Xc8e, Xt7, Xc7 ]
ynames = ['yt5b', 'yc5b', 'yt8e', 'yc8e', 'yt7', 'yc7' ]
yObjs = [yt5b, yc5b, yt8e, yc8e, yt7, yc7 ]

In [10]:
%mkdir best_models_v2

In [11]:
%cd best_models_v2

/Users/ak/Downloads/best_models_v2


In [22]:
%ls 

catboost_info/


In [28]:
#%rm Xc5b_BRAT.pkl

In [21]:
#%rm *.pkl

In [47]:
def cbc_score(predict_field, XX, yy):
    #display(XX.columns)
    cat_cols = ["flight_phase", "engine_family", "engine_type", "manufacturer",
        "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"]

    XX = XX.astype({'flight_datetime': 'datetime64[ns]'})
    XX.flight_datetime = pd.to_numeric(XX.flight_datetime)

    XX = XX.drop(columns=["engine_id", "aircraft_id"])

    y = yy[predict_field]
    
    X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.25, random_state=42)

    model = CatBoostRegressor(cat_features=cat_cols, early_stopping_rounds=500, silent=True)
    parameters = {'max_depth'    : [4, 6, 8],
                  'n_estimators'    : [1000, 3000],
                  'learning_rate' : [0.02, 0.1],
                  'l2_leaf_reg' : [3, 9]
                  }
                 
    Grid_CBC = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, refit=True, scoring= 'neg_mean_squared_error')
    Grid_CBC.fit(X_train, y_train)
    print('Parameter : ', predict_field)
    print("best score", abs(Grid_CBC.best_score_))
    print("\n The best parameters across ALL searched params:\n", Grid_CBC.best_params_)
    #pickle.dump( Grid_CBC.best_estimator_, open( exec(f"{count}.p"), "wb" ) )
    return [Grid_CBC.best_score_ , Grid_CBC.best_estimator_]

In [48]:
for i in range(6):
  print('Dataset : ', Xnames[i])
  scores = pd.DataFrame(columns=["parameter", "score"])
  exec(f"output_parameters = {ynames[i]}.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])")

  for param in tqdm(output_parameters):
      try:
          score = cbc_score(param, XObjs[i], yObjs[i])
          scores.loc[len(scores)] = [param, score[0]] 
          #print(score[1]
          with open(f"{Xnames[i]}_{param}.pkl", 'wb') as f:
            count_bytes = pickle.dump(score[1], f)
      except Exception as e:
          print(f'Skipped... {e}')
          continue
  print(scores)

Dataset :  Xt5b


  7%|▋         | 1/15 [03:07<43:40, 187.20s/it]

Parameter :  BRAT
best score 0.0010940170337297547

 The best parameters across ALL searched params:
 {'l2_leaf_reg': 9, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 3000}


 13%|█▎        | 2/15 [07:03<46:47, 215.93s/it]

Parameter :  DELFN
best score 0.1629234901237845

 The best parameters across ALL searched params:
 {'l2_leaf_reg': 3, 'learning_rate': 0.02, 'max_depth': 4, 'n_estimators': 3000}


...

Parameter :  ZPCN25_D
best score 0.0023545501501672688

 The best parameters across ALL searched params:
 {'l2_leaf_reg': 3, 'learning_rate': 0.02, 'max_depth': 8, 'n_estimators': 1000}


 95%|█████████▍| 18/19 [1:50:43<05:53, 353.82s/it]

Parameter :  ZT49_D
best score 5.891693115845143

 The best parameters across ALL searched params:
 {'l2_leaf_reg': 3, 'learning_rate': 0.02, 'max_depth': 8, 'n_estimators': 3000}


100%|██████████| 19/19 [1:57:12<00:00, 370.14s/it]

Parameter :  ZWF36_D
best score 64.32256504265925

 The best parameters across ALL searched params:
 {'l2_leaf_reg': 9, 'learning_rate': 0.02, 'max_depth': 8, 'n_estimators': 3000}
   parameter       score
0       DEGT   -0.903451
1      DPOIL   -0.000937
2       EGTC   -0.317855
3     EGTHDM   -4.566347
4     GEGTMC   -0.914319
5      GN2MC   -0.002143
6     GPCN25   -0.002111
7       GWFM   -0.161205
8      PCN12   -0.001401
9     PCN12I   -0.001401
10     PCN1K   -0.003614
11     PCN2C   -0.001168
12    SLOATL   -0.387477
13      WFMP -121.420754
14  ZPCN25_D   -0.002355
15    ZT49_D   -5.891693
16   ZWF36_D  -64.322565


In [49]:
with open('/content/best_models/Xt5b_BRAT.pkl','rb') as file:
        model = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/content/best_models/Xt5b_BRAT.pkl'

In [ ]:
model.predict(Xt5b)

In [ ]:
!zip -r weights.zip best_models